# Cleaning and Inserting technical indicators data into the final table: merged_df_4 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('AAPL_clean.csv')
df.head()
df["left_shoulder"] = df["High"].rolling(window=20).max()
df["right_shoulder"] = df["High"].rolling(window=20).max().shift(periods=-20)
df["head"] = df["High"].rolling(window=20).max().shift(periods=-10)

df["left_shoulder_diff"] = (df["head"] - df["left_shoulder"]) / df["head"]
df["right_shoulder_diff"] = (df["head"] - df["right_shoulder"]) / df["head"]

df["head_shoulders"] = ((df["left_shoulder_diff"] <= 0.05) & (df["right_shoulder_diff"] <= 0.05)).astype(int)
df.head()
HnS_df = df.iloc[19:10389].copy()

In [3]:
HnS_new_df = HnS_df.loc[:, ['Date','left_shoulder', 'right_shoulder', 'left_shoulder_diff', 'right_shoulder_diff', 'head_shoulders']]

In [4]:

df = pd.read_csv('AAPL_clean.csv')
# Calculate the 20-day SMA and standard deviation
df['SMA'] = df['Close'].rolling(window=20).mean()
df['STD'] = df['Close'].rolling(window=20).std()

# Calculate the upper and lower Bollinger Bands
df['UpperBand'] = df['SMA'] + (df['STD'] * 2)
df['LowerBand'] = df['SMA'] - (df['STD'] * 2)

df2 = pd.DataFrame()
df2 = df.iloc[19:].copy()
df2['Max_Min_diff'] = df2['High']-df2['Low']
df2['Open_Close_diff'] = df2['Open']-df2['Close']
df2 = df2[['Date', 'UpperBand', 'LowerBand', 'Max_Min_diff','Open_Close_diff', 'SMA']].copy()
df2 = df2.rename(columns={'UpperBand': 'BB_upperband'})
df2 = df2.rename(columns={'LowerBand': 'BB_lowerband'})
df2.head()
Bollinger_df = df2.copy()

In [5]:
merged_df = pd.merge(Bollinger_df, HnS_new_df, on='Date', how='inner')

In [6]:
merged_df.tail()

,Date,BB_upperband,BB_lowerband,Max_Min_diff,Open_Close_diff,SMA,left_shoulder,right_shoulder,left_shoulder_diff,right_shoulder_diff,head_shoulders
10365,2022-02-17,181.036470,157.983530,3.440003,2.149994,169.510000,176.649994,170.539993,0.000000,0.034588,1
10366,2022-02-18,180.846092,158.662908,4.349991,2.520004,169.754500,176.649994,168.910004,0.000000,0.043815,1
10367,2022-02-22,180.624198,159.154803,4.540008,0.659989,169.889500,176.649994,169.419998,0.000000,0.040928,1
10368,2022-02-23,180.581833,159.226169,6.399994,5.469986,169.904001,176.649994,172.639999,0.000000,0.022700,1
10369,2022-02-24,180.193545,159.919457,10.850006,-10.160003,170.056501,176.649994,174.139999,-0.006667,0.007636,1


In [7]:
df = pd.read_csv('AAPL_clean.csv')
df.head()

# KELTNER CHANNEL CALCULATION

def get_kc(high, low, close, kc_lookback, multiplier, atr_lookback):
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(alpha = 1/atr_lookback).mean()
    
    kc_middle = close.ewm(kc_lookback).mean()
    kc_upper = close.ewm(kc_lookback).mean() + multiplier * atr
    kc_lower = close.ewm(kc_lookback).mean() - multiplier * atr
    
    return kc_middle, kc_upper, kc_lower
kc = df
kc['kc_middle'], kc['kc_upper'], kc['kc_lower'] = get_kc(kc['High'], kc['Low'], kc['Close'], 20, 2, 10)
kc.tail(20)

kc_df = kc.loc[:, ['Date','kc_middle', 'kc_upper', 'kc_lower']]
kc_df.tail()

,Date,kc_middle,kc_upper,kc_lower
10404,2022-03-18,164.188274,174.130122,154.246425
10405,2022-03-21,164.245023,173.860688,154.629357
10406,2022-03-22,164.462879,174.018977,154.906781
10407,2022-03-23,164.736552,174.335041,155.138063
10408,2022-03-24,165.181002,174.605641,155.756364


In [8]:
kc_df['Date'] = pd.to_datetime(kc_df['Date'])
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df_2 = pd.merge(kc_df, merged_df, on='Date', how='inner')

In [9]:
merged_df_2.tail()

,Date,kc_middle,kc_upper,kc_lower,BB_upperband,BB_lowerband,Max_Min_diff,Open_Close_diff,SMA,left_shoulder,right_shoulder,left_shoulder_diff,right_shoulder_diff,head_shoulders
10365,2022-02-17,170.007667,178.210924,161.804409,181.036470,157.983530,3.440003,2.149994,169.510000,176.649994,170.539993,0.000000,0.034588,1
10366,2022-02-18,169.878730,178.131660,161.625800,180.846092,158.662908,4.349991,2.520004,169.754500,176.649994,168.910004,0.000000,0.043815,1
10367,2022-02-22,169.614029,178.071668,161.156391,180.624198,159.154803,4.540008,0.659989,169.889500,176.649994,169.419998,0.000000,0.040928,1
10368,2022-02-23,169.159552,178.051425,160.267678,180.581833,159.226169,6.399994,5.469986,169.904001,176.649994,172.639999,0.000000,0.022700,1
10369,2022-02-24,168.853859,179.026547,158.681172,180.193545,159.919457,10.850006,-10.160003,170.056501,176.649994,174.139999,-0.006667,0.007636,1


In [10]:
# Load the stock data from a CSV file
df = pd.read_csv("AAPL_Clean.csv")

# Calculate the price change between each day's closing price and the previous day's closing price
delta = df["Close"].diff()

# Define the time period for the RSI calculation
n = 14

# Calculate the average gain and loss for the first n days
gain = delta[1:n+1][delta[1:n+1] > 0].sum() / n
loss = -delta[1:n+1][delta[1:n+1] < 0].sum() / n

# Initialize the arrays to store the RSI values and the average gain and loss values
rsi = [100 * gain / (gain + loss)]
avg_gain = [gain]
avg_loss = [loss]

# Calculate the RSI values for the remaining days
for i in range(n+1, len(df)):
    change = delta[i]
    if change > 0:
        avg_gain.append((avg_gain[-1] * (n - 1) + change) / n)
        avg_loss.append((avg_loss[-1] * (n - 1)) / n)
    else:
        avg_gain.append((avg_gain[-1] * (n - 1)) / n)
        avg_loss.append((avg_loss[-1] * (n - 1) - change) / n)
    rs = avg_gain[-1] / avg_loss[-1]
    rsi.append(100 - 100 / (1 + rs))

# Slice the RSI array to match the length of the DataFrame
rsi = rsi[-len(df):]
df = df.drop(range(14))

# Add the RSI values to the DataFrame
df["RSI"] = rsi

rsi_df = df.copy()


In [11]:
rsi_df['Date'] = pd.to_datetime(rsi_df['Date'])
merged_df_3 = pd.merge(rsi_df, merged_df_2, on='Date', how='inner')

In [12]:
merged_df_3.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,RSI,kc_middle,kc_upper,...,BB_upperband,BB_lowerband,Max_Min_diff,Open_Close_diff,SMA,left_shoulder,right_shoulder,left_shoulder_diff,right_shoulder_diff,head_shoulders
10365,2022-02-17,171.029999,171.910004,168.470001,168.880005,168.880005,69589300,46.780097,170.007667,178.210924,...,181.036470,157.983530,3.440003,2.149994,169.510000,176.649994,170.539993,0.000000,0.034588,1
10366,2022-02-18,169.820007,170.539993,166.190002,167.300003,167.300003,82772700,44.462683,169.878730,178.131660,...,180.846092,158.662908,4.349991,2.520004,169.754500,176.649994,168.910004,0.000000,0.043815,1
10367,2022-02-22,164.979996,166.690002,162.149994,164.320007,164.320007,91162800,40.397843,169.614029,178.071668,...,180.624198,159.154803,4.540008,0.659989,169.889500,176.649994,169.419998,0.000000,0.040928,1
10368,2022-02-23,165.539993,166.149994,159.750000,160.070007,160.070007,90009200,35.423888,169.159552,178.051425,...,180.581833,159.226169,6.399994,5.469986,169.904001,176.649994,172.639999,0.000000,0.022700,1
10369,2022-02-24,152.580002,162.850006,152.000000,162.740005,162.740005,141147500,40.389504,168.853859,179.026547,...,180.193545,159.919457,10.850006,-10.160003,170.056501,176.649994,174.139999,-0.006667,0.007636,1


In [13]:
df = pd.read_csv('AAPL_Clean.csv')
df["MA"] = df["Close"].rolling(50).mean()
ma = df.loc[:,['Date','MA']]

ma_df = ma.loc[50:].copy()


ma_df['Date'] = pd.to_datetime(ma_df['Date'])
merged_df_4 = pd.merge(ma_df, merged_df_3, on='Date', how='inner')

In [14]:
Max_Min_diff = merged_df_4.pop('Max_Min_diff')
merged_df_4.insert(len(merged_df_4.columns), 'Max_Min_diff', Max_Min_diff)

Open_Close_diff = merged_df_4.pop('Open_Close_diff')
merged_df_4.insert(len(merged_df_4.columns), 'Open_Close_diff', Open_Close_diff)

merged_df_4.head()


merged_df_4.head()


,Date,MA,Open,High,Low,Close,Adj Close,Volume,RSI,kc_middle,...,BB_upperband,BB_lowerband,SMA,left_shoulder,right_shoulder,left_shoulder_diff,right_shoulder_diff,head_shoulders,Max_Min_diff,Open_Close_diff
0,1981-02-25,0.132266,0.112723,0.113281,0.112723,0.112723,0.088110,19488000,41.038379,0.125797,...,0.137111,0.103402,0.120257,0.138951,0.120536,-0.131818,0.018181,1,0.000558,0.000000
1,1981-02-26,0.132121,0.114397,0.114955,0.114397,0.114397,0.089418,10841600,42.859823,0.125208,...,0.133753,0.104360,0.119057,0.133929,0.120536,-0.090911,0.018181,1,0.000558,0.000000
2,1981-02-27,0.132232,0.118304,0.119420,0.118304,0.118304,0.092472,14761600,46.976857,0.124852,...,0.131364,0.105243,0.118304,0.128906,0.120536,-0.049997,0.018181,1,0.001116,0.000000
3,1981-03-02,0.132299,0.118862,0.119420,0.118862,0.118862,0.092908,11760000,47.558019,0.124545,...,0.130481,0.105401,0.117941,0.128906,0.120536,-0.049997,0.018181,1,0.000558,0.000000
4,1981-03-03,0.132266,0.117746,0.117746,0.117188,0.117188,0.091600,16172800,45.931538,0.124169,...,0.130393,0.105321,0.117857,0.128906,0.120536,-0.049997,0.018181,1,0.000558,0.000558


In [15]:
merged_df_4['Price_Movement'] = merged_df_4['Close'].diff().apply(lambda x: 1 if x > 0 else 0)
merged_df_4['Price_Movement'].iloc[0] = 0 # set the first value to 0
merged_df_4 = merged_df_4.iloc[30:].reset_index(drop=True)

/var/folders/0m/c46y49qj7z3ccyh9x502v6n40000gn/T/ipykernel_90616/47357971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_4['Price_Movement'].iloc[0] = 0 # set the first value to 0


In [16]:
merged_df_4 = merged_df_4.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume', 'Max_Min_diff', 'Open_Close_diff'], axis=1)
columns = merged_df_4.columns
print(columns)

Index(['Date', 'MA', 'Close', 'RSI', 'kc_middle', 'kc_upper', 'kc_lower',
       'BB_upperband', 'BB_lowerband', 'SMA', 'left_shoulder',
       'right_shoulder', 'left_shoulder_diff', 'right_shoulder_diff',
       'head_shoulders', 'Price_Movement'],
      dtype='object')


In [17]:
%store merged_df_4

Stored 'merged_df_4' (DataFrame)
